## 4.4.1 前処理

In [1]:
import numpy as np
import pandas as pd

### 欠損値への対応

In [2]:
# サンプルのデータセットを生成
df = pd.DataFrame(
    {
        'A': [1, np.nan, 3, 4, 5],
        'B': [6, 7, 8, np.nan, 10],
        'C': [11, 12, 13, 14, 15]
    }
)
df

,A,B,C
0,1.0,6.0,11
1,NaN,7.0,12
2,3.0,8.0,13
3,4.0,NaN,14
4,5.0,10.0,15


欠損値の除去

In [3]:
# 各要素が欠損値かどうかを確認
df.isnull()

,A,B,C
0,False,False,False
1,True,False,False
2,False,False,False
3,False,True,False
4,False,False,False


欠損値の補完

列Aの平均値
$$
\frac{1+3+4+5}{4}=3.25\tag{4.1}
$$

列Bの平均値
$$
\frac{6+7+8+10}{4}=7.75\tag{4.2}
$$

In [4]:
from sklearn.impute import SimpleImputer
# 平均値で欠損値を補完するインスタンスを生成
imp = SimpleImputer(strategy='mean')
# 欠損値を補完
imp.fit(df)
imp.transform(df)

array([[ 1.  ,  6.  , 11.  ],
       [ 3.25,  7.  , 12.  ],
       [ 3.  ,  8.  , 13.  ],
       [ 4.  ,  7.75, 14.  ],
       [ 5.  , 10.  , 15.  ]])

### カテゴリ変数のエンコーディング

In [5]:
df = pd.DataFrame(
    {
        'A': [1, 2, 3, 4, 5],
        'B': ['a', 'b', 'a', 'b', 'c']
    }
)
df

,A,B
0,1,a
1,2,b
2,3,a
3,4,b
4,5,c


カテゴリ変数のエンコーディング

In [6]:
from sklearn.preprocessing import LabelEncoder


# ラベルエンコーダのインスタンスを生成
le = LabelEncoder()
# ラベルのエンコーディング
le.fit(df.loc[:, 'B'])
le.transform(df.loc[:, 'B'])

array([0, 1, 0, 1, 2])

In [7]:
# 元の値を確認
le.classes_

array(['a', 'b', 'c'], dtype=object)

One-hotエンコーディング

In [8]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer


# DataFrameをコピー
df_ohe = df.copy()
# ラベルエンコーダのインスタンス化
le = LabelEncoder()
# 英語のa、b、cを1、2、3に変換
df_ohe.loc[:, 'B'] = le.fit_transform(df_ohe.loc[:, 'B'])
# One-hotエンコーダのインスタンス化
ohe = ColumnTransformer([("OneHotEncoder", OneHotEncoder(), [1])], remainder='passthrough')
# One-hotエンコーディング
df_ohe = ohe.fit_transform(df_ohe)
df_ohe

array([[1., 0., 0., 1.],
       [0., 1., 0., 2.],
       [1., 0., 0., 3.],
       [0., 1., 0., 4.],
       [0., 0., 1., 5.]])

### 特徴量の正規化

分散正規化

$$
x' = \frac{x-\mu}{\sigma} \tag{4.3}
$$

$x$: 特徴量  
$x'$: 分散正規化された特徴量  
$\mu$: $x'$の平均  
$\sigma$: 標準偏差

In [9]:
# DataFrameを作成
df = pd.DataFrame(
    {
        'A': [1, 2, 3, 4, 5],
        'B': [100, 200, 400, 500, 800]
    }
)
df

,A,B
0,1,100
1,2,200
2,3,400
3,4,500
4,5,800


分散正規化  

列Aの平均
$$
\frac{1+2+3+4+5}{5}=3 \tag{4.4}
$$

標準偏差
$$
\sqrt{\frac{1}{5}{(1-3)^2}+(2-3)^2+(3-3)^2+(4-3)^2+(5-3)^2}=\sqrt{2}=1.41421356 \tag{4.5}
$$

列Bの平均
$$
\frac{100+200+400+500+800}{5}=400 \tag{4.6}
$$

標準偏差
$$
\sqrt{\frac{1}{5}{(100-400)^2}+(200-400)^2+(400-400)^2+(500-400)^2+(800-400)^2}=\sqrt{60000}=244.94897428 \tag{4.7}
$$

列Aの2行目に分散正規化を行ったあとの値
$$
\frac{2-3}{\sqrt{2}}=-\frac{\sqrt{2}}{2}=-0.70710678  \tag{4.8}
$$
列Bの4行目に対して
$$
\frac{500-400}{\sqrt{60000}}=0.40824829  \tag{4.9}
$$

In [10]:
from sklearn.preprocessing import StandardScaler


# 分散正規化のインスタンスを生成
stdsc = StandardScaler()
# 分散正規化を実行
stdsc.fit(df)
stdsc.transform(df)

array([[-1.41421356, -1.22474487],
       [-0.70710678, -0.81649658],
       [ 0.        ,  0.        ],
       [ 0.70710678,  0.40824829],
       [ 1.41421356,  1.63299316]])

最小最大正規化  
**最小最大正規化**は、特徴量の最小量が0、最大値が1をとるように特徴量を正規化する処理である。
$$
x' = \frac{x-x_{min}}{x_{max}-x_{min}} \tag{4.10}
$$
$x$: 特徴量  
$x'$: 最小最大正規化された特徴量  
$x_{min}$: $x$の最小値
$x_{max}$: $x$の最大値

列Aの最小値は$1 (x_{min}=1)$、最大値は$5 (x_{max}=5)$  
列Bの最小値は$100 (y_{min}=100)$、最大値は$800 (y_{max}=800)$  

よって、列Aの2行目に最小最大正規化を行うと$\frac{2-1}{5-1}=0.25$、列Bの4行目に対しては$\frac{500-100}{800-100}=0.57142857$となる。

In [11]:
from sklearn.preprocessing import MinMaxScaler


# 最小最大正規化のインスタンスを生成
mmsc = MinMaxScaler()
# 最小最大正規化を実行
mmsc.fit(df)
mmsc.transform(df)

array([[0.        , 0.        ],
       [0.25      , 0.14285714],
       [0.5       , 0.42857143],
       [0.75      , 0.57142857],
       [1.        , 1.        ]])